# Tworzenie nowego popularity i sprzątanie danych

In [275]:
import os
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import time

In [276]:
folder_path = "../data_v2/data/"

files = os.listdir(folder_path)

raw_data = {}

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_json(file_path, lines=True)
    raw_data[file] = df

In [277]:
print(raw_data.keys())

dict_keys(['artists.jsonl', 'sessions.jsonl', 'tracks.jsonl', 'users.jsonl'])


In [278]:
sessions_df = raw_data['sessions.jsonl']
tracks_df = raw_data['tracks.jsonl']
artists_df = raw_data['artists.jsonl']

In [279]:
track_appearance_counts = sessions_df['track_id'].value_counts()
track_appearance_counts = track_appearance_counts.reindex(tracks_df['id'], fill_value=0)
track_appearance_counts.describe()

count    22412.000000
mean        64.433875
std         10.758655
min         23.000000
25%         57.000000
50%         64.000000
75%         72.000000
max        111.000000
Name: count, dtype: float64

Sessions w pełni pokrywa wszystkie utwory

In [280]:
event_map = {'play': 1, 'skip': -1, 'like': 2, 'advertisement': 0}

In [281]:
sessions_df['popularity_coefficient'] = sessions_df['event_type'].map(event_map)
popularity_per_track = sessions_df.groupby('track_id')['popularity_coefficient'].sum().reset_index()
popularity_per_track.rename(columns = {'popularity_coefficient':'popularity_from_sessions'}, inplace = True)

In [282]:
tracks_with_popularity = pd.merge(tracks_df, popularity_per_track, left_on='id', right_on='track_id', how='left')
# niepotrzebne ze względu na pełne pokrycie
# tracks_with_popularity['popularity_from_sessions'].fillna(0, inplace=True)

In [283]:
scaler = MinMaxScaler()
tracks_with_popularity[['normalized_popularity','normalized_popularity_from_sessions']] = scaler.fit_transform(tracks_with_popularity[['popularity','popularity_from_sessions']])

In [284]:
tracks_with_popularity[['normalized_popularity','normalized_popularity_from_sessions']].describe()

,normalized_popularity,normalized_popularity_from_sessions
count,22412.000000,22412.000000
mean,0.216158,0.378079
std,0.167672,0.122217
min,0.000000,0.000000
25%,0.083333,0.292683
50%,0.187500,0.378049
75%,0.333333,0.451220
max,1.000000,1.000000


In [285]:
fig = px.histogram(tracks_with_popularity, x=['normalized_popularity', 'normalized_popularity_from_sessions'],
                   nbins=50, opacity=0.7, barmode='overlay',
                   labels={'value': 'Normalized Popularity'},
                   title='Distribution of Normalized Popularity Values')
fig.show()

# Przygotowanie danych treningowych

In [286]:
tracks_with_popularity.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity_from_sessions,normalized_popularity,normalized_popularity_from_sessions
count,22412.000000,2.241200e+04,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000
mean,61.375602,2.301383e+05,0.166607,0.599096,0.648017,5.278824,-7.196227,0.082914,0.267758,0.040220,0.193141,0.518000,121.911945,49.002454,0.216158,0.378079
std,8.048246,7.209562e+04,0.372633,0.158259,0.209577,3.558813,3.738098,0.089317,0.276111,0.152828,0.167943,0.244477,29.615254,10.021829,0.167672,0.122217
min,51.000000,3.062200e+04,0.000000,0.000000,0.000103,0.000000,-44.410000,0.000000,0.000001,0.000000,0.012000,0.000000,0.000000,18.000000,0.000000,0.000000
25%,55.000000,1.914930e+05,0.000000,0.495000,0.508000,2.000000,-8.614500,0.033900,0.029800,0.000000,0.094400,0.325000,97.988000,42.000000,0.083333,0.292683
50%,60.000000,2.206670e+05,0.000000,0.608000,0.672000,5.000000,-6.393000,0.046300,0.165000,0.000003,0.125000,0.512000,120.041000,49.000000,0.187500,0.378049
75%,67.000000,2.562400e+05,0.000000,0.715000,0.816000,8.000000,-4.850000,0.086000,0.455000,0.000523,0.240000,0.713000,140.078000,55.000000,0.333333,0.451220
max,99.000000,4.120258e+06,1.000000,0.980000,0.999000,11.000000,0.642000,0.944000,0.996000,0.998000,0.997000,0.991000,220.099000,100.000000,1.000000,1.000000


In [287]:
tracks_with_popularity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22412 entries, 0 to 22411
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   22412 non-null  object 
 1   name                                 22412 non-null  object 
 2   popularity                           22412 non-null  int64  
 3   duration_ms                          22412 non-null  int64  
 4   explicit                             22412 non-null  int64  
 5   id_artist                            22412 non-null  object 
 6   release_date                         22412 non-null  object 
 7   danceability                         22412 non-null  float64
 8   energy                               22412 non-null  float64
 9   key                                  22412 non-null  int64  
 10  loudness                             22412 non-null  float64
 11  speechiness                 

In [288]:
tracks_with_popularity_artists = pd.merge(tracks_with_popularity, artists_df, left_on='id_artist', right_on='id', how='left')
tracks_with_popularity_artists.drop(columns=['track_id'], inplace=True)
tracks_with_popularity_artists.rename(columns = {'id_x':'track_id', 'name_x':'track_name', 'id_y':'artist_id', 'name_y':'artist_name'}, inplace = True)

In [289]:
tracks_with_popularity_artists.columns

Index(['track_id', 'track_name', 'popularity', 'duration_ms', 'explicit',
       'id_artist', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'popularity_from_sessions',
       'normalized_popularity', 'normalized_popularity_from_sessions',
       'artist_id', 'artist_name', 'genres'],
      dtype='object')

In [290]:
tracks_with_popularity_artists.drop(columns=['id_artist', 'normalized_popularity', 'normalized_popularity_from_sessions'], axis=1, inplace=True)

In [291]:
tracks_with_popularity_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22412 entries, 0 to 22411
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  22412 non-null  object 
 1   track_name                22412 non-null  object 
 2   popularity                22412 non-null  int64  
 3   duration_ms               22412 non-null  int64  
 4   explicit                  22412 non-null  int64  
 5   release_date              22412 non-null  object 
 6   danceability              22412 non-null  float64
 7   energy                    22412 non-null  float64
 8   key                       22412 non-null  int64  
 9   loudness                  22412 non-null  float64
 10  speechiness               22412 non-null  float64
 11  acousticness              22412 non-null  float64
 12  instrumentalness          22412 non-null  float64
 13  liveness                  22412 non-null  float64
 14  valenc

# Konwersja daty

In [292]:
tracks_with_popularity_artists['release_date'] = pd.to_datetime(tracks_with_popularity_artists['release_date'], format='mixed')
tracks_with_popularity_artists['release_date_numeric'] = tracks_with_popularity_artists['release_date'].apply(lambda x: time.mktime(x.timetuple()) if not pd.isnull(x) else None)

In [293]:
tracks_with_popularity_artists[['release_date', 'release_date_numeric']].describe()

,release_date,release_date_numeric
count,22412,2.241200e+04
mean,2006-08-04 16:57:32.864536832,1.154705e+09
min,1929-01-01 00:00:00,-1.293844e+09
25%,2000-05-16 00:00:00,9.584280e+08
50%,2011-02-01 00:00:00,1.296515e+09
75%,2017-05-12 00:00:00,1.494540e+09
max,2021-04-10 00:00:00,1.618006e+09
std,NaN,4.404159e+08


TODO One hot encoding

In [294]:
# one_hot_genres = tracks_with_popularity_artists['genres'].str.get_dummies(', ')
# tracks_with_popularity_artists_one_hot_genres = pd.concat([tracks_with_popularity_artists, one_hot_genres], axis=1)

# Normalizacja

In [295]:
tracks_with_popularity_artists.columns

Index(['track_id', 'track_name', 'popularity', 'duration_ms', 'explicit',
       'release_date', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'popularity_from_sessions', 'artist_id',
       'artist_name', 'genres', 'release_date_numeric'],
      dtype='object')

In [296]:
numerical_columns = ['popularity', 'duration_ms', 'explicit',
                      'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'popularity_from_sessions', 'release_date_numeric']

In [297]:
tracks_with_popularity_artists[numerical_columns].isna().sum()

popularity                  0
duration_ms                 0
explicit                    0
danceability                0
energy                      0
key                         0
loudness                    0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
popularity_from_sessions    0
release_date_numeric        0
dtype: int64

In [298]:
scaler = MinMaxScaler()
tracks_with_popularity_artists_scaled = tracks_with_popularity_artists.copy()
tracks_with_popularity_artists_scaled[numerical_columns] = scaler.fit_transform(tracks_with_popularity_artists[numerical_columns])

In [301]:
tracks_with_popularity_artists_scaled.describe()

,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity_from_sessions,release_date_numeric
count,22412.000000,22412.000000,22412.000000,22412,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000
mean,0.216158,0.048786,0.166607,2006-08-04 16:57:32.864536832,0.611323,0.648630,0.479893,0.826018,0.087833,0.268832,0.040301,0.183900,0.522705,0.553896,0.378079,0.840891
min,0.000000,0.000000,0.000000,1929-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.083333,0.039336,0.000000,2000-05-16 00:00:00,0.505102,0.508458,0.181818,0.794537,0.035911,0.029918,0.000000,0.083655,0.327952,0.445200,0.292683,0.773485
50%,0.187500,0.046470,0.000000,2011-02-01 00:00:00,0.620408,0.672639,0.454545,0.843847,0.049047,0.165661,0.000003,0.114721,0.516650,0.545395,0.378049,0.889592
75%,0.333333,0.055168,0.000000,2017-05-12 00:00:00,0.729592,0.816798,0.727273,0.878096,0.091102,0.456827,0.000524,0.231472,0.719475,0.636432,0.451220,0.957599
max,1.000000,1.000000,1.000000,2021-04-10 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,0.167672,0.017629,0.372633,NaN,0.161488,0.209809,0.323528,0.082973,0.094616,0.277220,0.153134,0.170501,0.246697,0.134554,0.122217,0.151250


: 

In [300]:
tracks_with_popularity_artists_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22412 entries, 0 to 22411
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   track_id                  22412 non-null  object        
 1   track_name                22412 non-null  object        
 2   popularity                22412 non-null  float64       
 3   duration_ms               22412 non-null  float64       
 4   explicit                  22412 non-null  float64       
 5   release_date              22412 non-null  datetime64[ns]
 6   danceability              22412 non-null  float64       
 7   energy                    22412 non-null  float64       
 8   key                       22412 non-null  float64       
 9   loudness                  22412 non-null  float64       
 10  speechiness               22412 non-null  float64       
 11  acousticness              22412 non-null  float64       
 12  instrumentalness  